# Data processing

Explore later: (https://github.com/dunovank/jupyter-themes)


In [1]:
import pandas as pd
import os
from datetime import datetime
from datetime import timedelta
import numpy as np
import re

In [2]:
def load_incidents(filename):
    #read csv
    csv_path = os.path.join("data/traffic_incidents", filename)
    df = pd.read_csv(csv_path)
    #drop nan values, plus useless columns
    df = df.dropna()
    df = df.drop(["año_cierre", "mes_cierre", "mes"], axis=1)
    
    #drop badly recorded data
    miss = []
    r = re.compile('.{2}:.{2}:.{2}')
    for i, row in df.iterrows():
        if (r.match(row["hora_creacion"])) and (r.match(row["hora_cierre"])):
            continue
        else:
            miss.append(i)
        
    df = df.drop(miss)
    
    #renaming columns
    df = df.rename(columns={
        "fecha_creacion" : "start_date",
        "hora_creacion" : "start_hour",
        "dia_semana" : "weekday",
        "codigo_cierre" : "incident_code",
        "fecha_cierre" : "end_date",
        "hora_cierre" : "end_hour",
        "delegacion_inicio" : "start_county",
        "incidente_c4" : "type_incident",
        "latitud" : "lat",
        "longitud" : "lng",
        "clas_con_f_alarma" : "class_incident",
        "tipo_entrada" : "aid_channel",
        "delegacion_cierre" : "end_county" 
    })
    #renaming county values
    for col in ["start_county", "end_county"]:
        df = df.replace({col: {"GUSTAVO A. MADERO": "GAM",
                               "ALVARO OBREGON": "AOB",
                              "CUAJIMALPA" : "CUA",
                              "IZTAPALAPA" : "IZP",
                              "AZCAPOTZALCO" : "AZC",
                              "TLALPAN" : "TLA",
                              "MIGUEL HIDALGO" : "MHI",
                              "CUAUHTEMOC" : "CUA",
                              "IZTACALCO" : "IZC",
                              "VENUSTIANO CARRANZA" : "VCA",
                              "MAGDALENA CONTRERAS" : "MCO",
                              "BENITO JUAREZ" : "BJU",
                              "COYOACAN" : "COY",
                              "XOCHIMILCO" : "XOC",
                              "TLAHUAC" : "TLH",
                              "MILPA ALTA" : "MAL"}
                        })
    #renaming weekdays
    df = df.replace({"weekday":{"Lunes" : "MO",
                               "Martes" : "TU",
                               "Miércoles" : "WE",
                               "Jueves" : "TH",
                               "Viernes" : "FR",
                               "Sábado" : "SA",
                               "Domingo" : "SU"}})
    #renaming incident code
    def change_code(x):
        if "(D)" in x:
            return "duplicated"
        elif "(N)" in x:
            return "not found"
        elif "(A)" in x:
            return "affirmative"
        elif "(F)" in x:
            return "false"
        else:
            return "informative"
    
    df["incident_code"] = df["incident_code"].apply(change_code)
    
    #renaming class_incident
    df = df.replace({"class_incident":{"EMERGENCIA" : "Emergencies",
                               "URGENCIAS MEDICAS" : "Medical emergencies",
                               "FALSA ALARMA" : "False alarm",
                               "DELITO" : "Crime"}
                    })
    #renaming aid_channel
    df = df.replace({"aid_channel":{"LLAMADA DEL 911" : "911 call",
                               "LLAMADA APP911" : "911 App",
                               "BOTÓN DE AUXILIO" : "Help button",
                               "RADIO" : "Radio",
                               "ZELLO" : "Zello",
                               "REDES" : "Social media",
                               "CÁMARA" : "Camera"}
                    })
    #creating new column: time_response
    FMT = "%H:%M:%S"
    for i, row in df.iterrows():
        time1 = datetime.strptime(row["start_hour"], FMT)
        time2 = datetime.strptime(row["end_hour"], FMT)
        tdelta = (time2 - time1).total_seconds() // 60

        if tdelta < 0:
            tdelta = -tdelta
        df.at[i, "time_response"] = tdelta
      
    return df

In [3]:
df17 = load_incidents("incidents_2017.csv")
df18 = load_incidents("incidents_2018.csv")
df19 = load_incidents("incidents_2019.csv")
df20 = load_incidents("incidents_2020.csv")

# Concatenate all dfs

In [4]:
CDMX_1720 = pd.concat([df17, df18, df19, df20], ignore_index=True)

In [5]:
CDMX_1720.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761101 entries, 0 to 761100
Data columns (total 16 columns):
folio             761101 non-null object
start_date        761101 non-null object
start_hour        761101 non-null object
weekday           761101 non-null object
incident_code     761101 non-null object
end_date          761101 non-null object
end_hour          761101 non-null object
start_county      761101 non-null object
type_incident     761101 non-null object
lat               761101 non-null float64
lng               761101 non-null float64
class_incident    761101 non-null object
aid_channel       761101 non-null object
end_county        761101 non-null object
geopoint          761101 non-null object
time_response     761101 non-null float64
dtypes: float64(3), object(13)
memory usage: 92.9+ MB


In [6]:
CDMX_1720.to_csv("cdmx_c5_17to20.csv")

# Incident rate table

In [7]:
vfleet = pd.read_csv("data/vehicle_fleet/fleet_2018.csv")

In [8]:
vfleet = vfleet.drop(["PROD_EST", "COBERTURA", "ESTATUS"], axis=1)

In [9]:
#keep only records from CDMX
dprows = []
for i, row in vfleet.iterrows():
    if row["ID_ENTIDAD"] != 9:
        dprows.append(i)

vfleet = vfleet.drop(dprows)

In [10]:
#get total vechicle fleet per county
sum_cols = list(vfleet)

del sum_cols[0:3]

vfleet['total_fleet'] = vfleet[sum_cols].sum(axis=1)

vfleet

,ANIO,ID_ENTIDAD,ID_MUNICIPIO,AUTO_OFICIAL,AUTO_PUBLICO,AUTO_PARTICULAR,CAM_PAS_OFICIAL,CAM_PAS_PUBLICO,CAM_PAS_PARTICULAR,CYC_CARGA_OFICIAL,CYC_CARGA_PUBLICO,CYC_CARGA_PARTICULAR,MOTO_OFICIAL,MOTO_DE_ALQUILER,MOTO_PARTICULAR,total_fleet
268,2018,9,2,0,7836,279570,0,0,1201,0,203,5201,0,0,17299,311310
269,2018,9,3,0,11376,449980,0,0,2279,0,139,3669,0,0,30830,498273
270,2018,9,4,0,1790,144290,0,0,779,0,46,1355,0,0,8779,157039
271,2018,9,5,0,22268,519866,0,0,3971,0,217,5474,0,0,36823,588619
272,2018,9,6,0,8954,223450,0,0,1649,0,151,3811,0,0,15670,253685
273,2018,9,7,0,23466,673711,0,1885,7747,0,321,8448,0,0,58015,773593
274,2018,9,8,0,3584,119389,0,0,148,0,18,509,0,0,8404,132052
275,2018,9,9,0,1626,49616,0,0,451,0,11,295,0,0,3757,55756
276,2018,9,10,0,10548,380747,0,0,2654,0,235,6294,0,0,34498,434976
277,2018,9,11,0,3846,113230,0,0,1321,0,31,798,0,0,16381,135607


In [11]:
#renaming column values
vfleet = vfleet.replace({"ID_MUNICIPIO": {5 : "GAM",
                               10 : "AOB",
                              4 : "CUA",
                              7 : "IZP",
                              2: "AZC",
                              12 : "TLA",
                              16 : "MHI",
                              15 : "CUA",
                              6 : "IZC",
                              17 : "VCA",
                              8 : "MCO",
                              14 : "BJU",
                              3 : "COY",
                              13 : "XOC",
                              11 : "TLH",
                              9 : "MAL"}
                        })

In [12]:
#create copy of df18 and drop duplicated reports
unique18 = df18.copy()

dup = []
for i, row in unique18.iterrows():
    if row["incident_code"] == "duplicated":
        dup.append(i)

unique18 = unique18.drop(dup)

In [13]:
#get total of incicents per county
incidents_county18 = unique18.groupby("start_county").count()["folio"]

total_incidents18 = pd.DataFrame({"total_incident": incidents_county18})

In [14]:
rate_incident = vfleet.merge(total_incidents18, left_on="ID_MUNICIPIO", right_on="start_county")

In [15]:
#calculate incident rate for every 10k vehicles, per county
for i,row in rate_incident.iterrows():
    rate_incident.at[i, "rate_incident"] = row["total_incident"] / (row["total_fleet"] / 10000)

In [16]:
rate_incident

,ANIO,ID_ENTIDAD,ID_MUNICIPIO,AUTO_OFICIAL,AUTO_PUBLICO,AUTO_PARTICULAR,CAM_PAS_OFICIAL,CAM_PAS_PUBLICO,CAM_PAS_PARTICULAR,CYC_CARGA_OFICIAL,CYC_CARGA_PUBLICO,CYC_CARGA_PARTICULAR,MOTO_OFICIAL,MOTO_DE_ALQUILER,MOTO_PARTICULAR,total_fleet,total_incident,rate_incident
0,2018,9,AZC,0,7836,279570,0,0,1201,0,203,5201,0,0,17299,311310,5367,172.400501
1,2018,9,COY,0,11376,449980,0,0,2279,0,139,3669,0,0,30830,498273,10592,212.574231
2,2018,9,CUA,0,1790,144290,0,0,779,0,46,1355,0,0,8779,157039,15873,1010.768026
3,2018,9,CUA,0,7840,461185,0,390,1886,0,711,18673,0,0,43959,534644,15873,296.889145
4,2018,9,GAM,0,22268,519866,0,0,3971,0,217,5474,0,0,36823,588619,13224,224.661453
5,2018,9,IZC,0,8954,223450,0,0,1649,0,151,3811,0,0,15670,253685,5392,212.547056
6,2018,9,IZP,0,23466,673711,0,1885,7747,0,321,8448,0,0,58015,773593,19507,252.161020
7,2018,9,MCO,0,3584,119389,0,0,148,0,18,509,0,0,8404,132052,2126,160.997183
8,2018,9,MAL,0,1626,49616,0,0,451,0,11,295,0,0,3757,55756,832,149.221608
9,2018,9,AOB,0,10548,380747,0,0,2654,0,235,6294,0,0,34498,434976,9606,220.839770


In [17]:
rate_incident.to_csv("rate_incident18.csv")

# XXXXXXXXXXXXXXXXXXXXXXX

__REFERENCES__

https://pymotw.com/2/datetime/#:~:text=You%20can%20use%20datetime%20to,days%2C%20seconds%2C%20and%20microseconds.

Calculate time intervals
https://stackoverflow.com/questions/3096953/how-to-calculate-the-time-interval-between-two-time-strings

Convert time object into total minutes
https://stackoverflow.com/questions/27475583/how-to-convert-time-object-into-total-minutes-python

Assignate column values while iterating (df.at[i,"column_name"])
https://stackoverflow.com/questions/23330654/update-a-dataframe-in-pandas-while-iterating-row-by-row

Check if strings follow a format
https://stackoverflow.com/questions/14966647/check-python-string-format